# Covid-19 Lab Notebook

(to edit this notebook and the associated python files, `git checkout` the corresponding commit by its hash, eg. `git checkout 422024d`)

In [1]:
from IPython.display import display, Markdown
from datetime import datetime
cur_datetime = datetime.now()
display(Markdown(f'# {cur_datetime.strftime("%d/%b/%Y %H:%M")}'))

# 25/Jun/2020 11:16

# Covid-19 model dynamics on SD

On this notebook, we'll explore and visualize multiple models of covid-19 behavior in a dynamic system approach. The models were written on cadCAD - a library for Complex Adaptive Dynamics simulations which allows you to mix and prototype 
different modelling paradigms in a reproducible and consistent manner.

Differently from other data processing methods, e.g. machine learning models, where the objective is to predict outputs based on previous information, working with cadCAD allows us to pursue a different goal. As it works with dynamic simulations, we aim to create a range of possible future scenarios for Covid-19's spread according to the behavior of epidemiological parameters.

In [2]:
%%capture
%matplotlib inline

# Dependences
from time import time
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

# Experiments
import run

In [3]:
# Run all experiments. Typical run duration for an Core-i3 laptop is about 2-3min.
# Tweak the prey_predator_abm/sim_params.py file if you want it to take longer (or not).
start_time = time()
experiments = run.run()
end_time = time()
print("Execution in {:.1f}s".format(end_time - start_time))

Configurations Length: 3
Execution Method: parallelize_simulations
Execution Mode: parallelized


ValueError: too many values to unpack (expected 2)

# Covid-19 SIR model

In order to create a mathematical representation of Covid-19's spread, we will use compartmental models of infectious diseases. These models assign the system's population to compartments with labels that represent different states where the individuals can be, and the order of the labels shows the flow patterns between these compartments. As described in Wikipedia,

> "[Compartmental] models try to predict things such as how a disease spreads, or the total number infected, or the duration of an epidemic, and to estimate various epidemiological parameters such as the reproductive number. Such models can show how different public health interventions may affect the outcome of the epidemic, e.g., what the most efficient technique is for issuing a limited number of vaccines in a given population."

You can find more information about compartmental models on Wikipedia's page: https://en.wikipedia.org/wiki/Compartmental_models_in_epidemiology.

The simplest compartmental model is SIR, which consists of three compartments:
- **S**usceptible: The number of individuals at risk of contracting the disease;
- **I**nfectious: The number of individuals who have been infected and are capable of infecting susceptible individuals;
- **R**ecovered: The number of individuals who have been infected and have recovered from the disease.

As SIR is a very simple model, **it considers that the disease's death rate is negligible**.

The SIR model can be expressed by the following ordinary differential equations:

\begin{aligned}{\frac {d}{dt}Susceptible}&= - \beta * Infected * {\frac {Susceptible}{Total Population}}
\\{\frac{d}{dt}Infected}&=\beta * Infected * \frac {Susceptible}{Total Population} -\gamma * Infected
\\{\frac {d}{dt}Recovered}&=\gamma *Infected\end{aligned}

Where the parameters are:
- **Infection rate** ($\beta$): expected amount of people an infected person infects per day
- **Recovering rate** ($\gamma$): the proportion of infected recovering per day ($\gamma$ = 1 / recovery time in days)

The ratio between $\beta$ and $\gamma$ results on the **basic reproductive number ($R₀$)**, a very important parameter that determines **how contagious is a disease**. As said in Wikipedia,

> "[The reproductive number] of an infection can be thought of as the expected number of cases directly generated by one case in a population where all individuals are susceptible to infection. The definition describes the state where no other individuals are infected or immunized (naturally or through vaccination)."

You can find more information about $R₀$ on Wikipedia's page: https://en.wikipedia.org/wiki/Basic_reproduction_number.

In the first model built, the definition of parameters is up to the user and they are not time dependent, i.e., we have a deterministic system. This means that if we make a simulation with the same initial populations and parameters 100 times, we will get 100 equal results.

Evidently, this is not how real world works and it will not allow us to reach our goal of creating multiple spread scenarios, so it's quite important that stochasticity is added to the model.

## SIR model stock and flow diagram
Before properly building the model, it's essential to graphically represent it so that our problem is conceptually understood.

The following stock and flow diagram shows how the information flows through the SIR model stocks of populations.

![covid-19 SIR model mechanism](images/covid-19_sir_model_stock_flow.png)

## Covid-19 SIR model mechanism

As we are initially building a model with considerably low complexity, we will not consider exogenous process (at least for now). One example of exogenous process that impacts Covid-19's populations is the availability of hospital beds. If the hospitals in some certain location are overcrowded, the disease's death rate may increase.

In this model, the behaviors of the SIR model are simply going to be **infected and recovered growth**, as those are the two phenomena controlled by us (as seen in the equations, the susceptible population only changes by infected growth, so there's no behavior related to susceptible growth).

The infected growth causes a decrease in the susceptible population, as previously susceptible individuals became infected; for obvious reasons, it also increases the infected population.

Similarly, the recovered growth causes a decrease in the infected population, as previously infected individuals became recovered, and it increases the recovered population.

![covid-19 SIR model mechanism](images/covid-19_sir_model.png)

The model's simulation was created given the following parameters and initial state variables:
- **Infection rate** ( $\beta$ ): 0.5 (this means an infected individual infects other individual every two days);
- **Recovering rate** ($\gamma$): 0.07 (as 𝛾 = 1 / recovery time, this parameter is calculated using an average recovery time of 14 days. So, $\gamma$ = 1 / 14 $\approx$ 0.07);
- **Reproductive number** ($R₀$): $\beta$ / $\gamma$ $\approx$ 7.14; 
- **Susceptible population**: 990;
- **Infected population**: 10;
- **Recovered population**: 0.

When plotting a graph of the population evolution over time, some relevant observations can be done:
- In the first 20 timesteps, the susceptible population decreases drastically until it reaches zero;
- The infected population grows until it reaches its peak a little before timestep 20. Afterwards, it decreases until it reaches zero;
- The recovered population doesn't grow at such high rates in the beggining, which is explained by the average recovering time used in $\gamma$. From timestep 15, it acquires a logarithmic tendency, until all the population is considered recovered.

The system's behavior is close to expected, as susceptible population's low matches infected population's peak and recovered population's gets close to its peak around 15 days after infected's peak.

In [1]:
df = experiments.dataset[0] 
plt.rcParams["figure.figsize"]=20,5
fig, ax = plt.subplots()
ax.plot(df['timestep'], df['susceptible'], label='susceptible'  )
ax.plot(df['timestep'], df['infected'], label='infected'   )
ax.plot(df['timestep'], df['recovered'], label='recovered')
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])

# Put a legend to the right of the current axis
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
ax.set_xlabel("Time (days)")
ax.set_ylabel("People")
plt.show()

NameError: name 'experiments' is not defined

# Covid-19 SEIR model

The SIR model can give us a decent analysis on the behavior of an infectious disease, but its simplicity limitates it. As many infections, Covid-19 has a significant incubation period during which individuals have been infected but neither show symptoms nor are capable of infecting other individuals. Because of that, the **SEIR** model can represent it in a better way.


The SEIR model is described by the following equations:

\begin{aligned}{\frac {d}{dt}Susceptible}&= - \beta * Infected * {\frac {Susceptible}{Total Population}}
\\{\frac {d}{dt}Exposed}&=\beta * Infected * {\frac {Susceptible}{Total Population}} - \delta * Exposed 
\\{\frac{d}{dt}Infected}&=\delta * Exposed - \gamma * Infected
\\{\frac {d}{dt}Recovered}&=\gamma*Infected
\end{aligned}

Where the parameters are:
- **Infection rate** ($\beta$): expected amount of people an infected person infects per day
- **Recovering rate** ($\gamma$): the proportion of infected people recovering per day ($\gamma$ = 1 / recovery time in days)
- **Exposure rate** ($\delta$): expected rate that exposed people turn into infected ($\delta$ = 1 / incubation period in days)

As seen above, SEIR also considers the disease's death rate negligible. It's also notable that it has very similar equations to SIR. Besides adding an equation that represents the exposed population, SEIR also has differences in the infected population equation.

While SIR considered the difference between **the amount of individuals infected by and infected person** and the amount of infected individuals recovering per day, SEIR considers the difference between **the amount of exposed individuals turned into infected** and the amount of infected individuals recovering per day.

When conceptually analyzing the meaning of each compartment, it becomes very simple to understand this change in the equation.


## Covid-19 SEIR model stock and flow diagram 

Differently from SIR model, the population in SEIR doesn't flow directly from the susceptible stock to the infected one. Firstly, the exposed growth makes them flow to the exposed stock; after the incubation period, the infected growth makes them flow to the infected stock and, after the recovery time, the population finally flows to the recovered stock.



![covid-19 SEIR model mechanism](images/covid-19_seir_model_stock_flow.png)

## Covid-19 SEIR model mechanism

As seen in the equations and the stock and flow diagram, the difference between both models' mechanisms is that SEIR has an exposed growth behavior that triggers a decrease in the susceptible population and an increase in the exposed population. Besides that, infected growth causes a decrease in the exposed population, differently from SIR, where it decreased the susceptible one. Exogenous processes still weren't added to the model.

![covid-19 SIR model mechanism](images/covid-19_seir_model.png)

The model's simulation was created given the following parameters and initial state variables:
- **Infection rate** ( $\beta$ ): 1 (this means an infected individual infects other individual once a day);
- **Recovering rate** ( $\gamma$ ): 0.25 (differently from SIR model, this parameter is calculated using an average recovery time of 4 days. So, $\gamma$ = 1 / 4 = 0.25);
- **Reproductive number** ($R₀$): $\beta$ / $\gamma$ = 4; 
- **Exposure rate** ($\delta$): 0.333 (as $\delta$ = 1 / incubation period in days, this parameter is calculated using an average incubation period of 3 days. So, $\delta$ = 1 / 3 $\approx$ 0.333;
- **Susceptible population**: 9990;
- **Exposed population**: 10;
- **Infected population**: 0;
- **Recovered population**: 0.

As we can see, the parameters are very different from the ones used in SIR's simulation. This was made in order to see the difference this parameters make on the system's behavior. As said in the beggining of the notebook, the differential of using cadCAD is the ability to create varied scenarios of a system.

The graph shows some notable differences from the one shown in SIR model:
- The susceptible population decreases at low rate for quite longer than on SIR, only begginig to decrease considerably a little before the 20th day and approaching its low around the 35th day;
- The exposed population reaches higher rates approximately at the same time that susceptible population begins to decrease, but because of the high $\delta$ used, the curve is quite flat;
- The infected population shows a very similar behavior to exposed's one, with a little "delay" (justified by the incubation period) and a higher peak (justified by the lower recovering rate than the exposure rate). It is also much flatter than the presented on SIR, which shows the influence of $\gamma$ on the system's behavior;
- The recovered population reaches its peak considerably before than on SIR, with a much higher growth rate between timesteps 20 and 40.

In [ ]:
df = experiments.dataset[1] 
fig, ax = plt.subplots()
plt.rcParams["figure.figsize"]=20,5
ax.plot(df['timestep'], df['exposed'], label='exposed')
ax.plot(df['timestep'], df['susceptible'], label='susceptible')
ax.plot(df['timestep'], df['infected'], label='infected'   )
ax.plot(df['timestep'], df['recovered'], label='recovered')
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])

# Put a legend to the right of the current axis
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

ax.set_xlabel("Time (days)")
ax.set_ylabel("People")
plt.show()

# Covid-19 SEIRD model

As we know, Covid-19 is a disease with a significant death rate (around 5% worldwide). Because of that, SIR and SEIR models can be considerably inaccurate. Therefore, the SEIRD model can better represent Covid-19, as it includes individuals who died because of the disease in compartment **D**.

The SEIRD model is based on the following equations:

\begin{aligned}{\frac {d}{dt}Susceptible}&= - \beta * Infected * {\frac {Susceptible}{Total Population}}
\\{\frac {d}{dt}Exposed}&=\beta * Infected * {\frac {Susceptible}{Total Population}} - \delta * Exposed 
\\{\frac{d}{dt}Infected}&=\delta * Exposed - (1 - \alpha) * \gamma * Infected - \alpha * \rho * Infected
\\{\frac {d}{dt}Recovered}&= (1 - \alpha) * \gamma *Infected
\\{\frac {d}{dt}Dead}&=\alpha * \rho * Infected\end{aligned}

Where the parameters are:
- **Infection rate** ($\beta$): expected amount of people an infected person infects per day
- **Recovering rate** ($\gamma$): the proportion of infected people recovering per day ($\gamma$ = 1 / recovery time in days)
- **Exposure rate** ($\delta$): expected rate that exposed people turn into infected ($\delta$ = 1 / incubation period in days)
- **Death proportion rate** ($\rho$): rate at which infected people die per day ($\rho$ = 1 / days from infection until death)
- **Death rate** ($\alpha$): the probability an infected person has of dying

We can again see that the equations are similar to the previous models. Besides including a death population equation, the differences occur in the infected and recovered ones, where **death rate is now considered**.

## Covid-19 SEIRD model stock and flow diagram 

The stock and flow diagram shows that, in SEIRD model, the population in the infected stock can flow to two population stocks (dead or recovered).

![covid-19 SEIRD model mechanism](images/covid-19_seird_model_stock_flow.png)

## Covid-19 SEIRD model mechanism 

The difference between SEIR and SEIRD mechanisms is simply the addition of a death growth behavior that increases the dead population and decreases the infected population. It's interesting to observe that on SEIRD, we have a mechanism (infected population) that is triggered by three different behaviors, which shows cadCAD'S potential of adding much complexity to a system.

![covid-19 SEIRD model mechanism](images/covid-19_seird_model.png)

The model's simulation was created given the following parameters and initial state variables:
- **Infection rate** ( $\beta$ ): 1;
- **Recovering rate** ( $\gamma$ ): 0.25; 
- **Exposure rate** ($\delta$): 0.333;
- **Death proportion rate** ($\rho$): 0.111 (as $\rho$ = 1 / days from infection until death, this parameter is calculated considering an average time from infection until death of 9 days, so $\rho$ = 1 / 9 $\approx$ 0.111;
- **Death rate** ($\alpha$): 0.01 (this means 1% of infected people will die);
- **Susceptible population**: 9990;
- **Exposed population**: 10;
- **Infected population**: 0;
- **Recovered population**: 0;
- **Dead population**: 0.

In [ ]:
df = experiments.dataset[2] 
fig, ax = plt.subplots()
plt.rcParams["figure.figsize"]=20,5
ax.plot(df['timestep'], df['exposed'], label='exposed')
ax.plot(df['timestep'], df['susceptible'], label='susceptible')
ax.plot(df['timestep'], df['infected'], label='infected'   )
ax.plot(df['timestep'], df['recovered'], label='recovered')
ax.plot(df['timestep'], df['dead'], label='dead')
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])

# Put a legend to the right of the current axis
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

ax.set_xlabel("Time (days)")
ax.set_ylabel("People")
plt.show()

### Covid-19 SEIRD model with time variant R₀

This is the standard introductory model for Covid-19 spread. It is based on the following equations:

\begin{aligned}{\frac {d}{dt}Susceptible}&= - \beta * Infected * {\frac {Susceptible}{Total Population}}
\\{\frac {d}{dt}Exposed}&=\beta * Infected * {\frac {Susceptible}{Total Population}} - \delta * Exposed 
\\{\frac{d}{dt}Infected}&=\delta * Exposed - (1 - \alpha) * \gamma * Infected - \alpha * \rho * Infected
\\{\frac {d}{dt}Recovered}&= (1 - \alpha) * \gamma *Infected
\\{\frac {d}{dt}Dead}&=\alpha * \rho * Infected\end{aligned}

Where the parameters are:
- $\beta$ expected amount of people an infected person infects per day
- $\gamma$ the proportion of infected recovering per day ($\gamma$ = 1 / D)
- $\delta$ expected rate that exposed people turn into infected
- $\rho$ rate at wich infected people die per day ($\rho$ = 1 / D)
- $\alpha$ death probability
- $R₀$: the total number of people an infected person infects (R₀ = β / γ)


In [ ]:
df = experiments.dataset[3] 
fig, ax = plt.subplots()
plt.rcParams["figure.figsize"]=20,5
ax.plot(df['timestep'], df['exposed'], label='exposed')
ax.plot(df['timestep'], df['susceptible'], label='susceptible')
ax.plot(df['timestep'], df['infected'], label='infected'   )
ax.plot(df['timestep'], df['recovered'], label='recovered')
ax.plot(df['timestep'], df['dead'], label='dead')
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])

# Put a legend to the right of the current axis
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

ax.set_xlabel("Time (days)")
ax.set_ylabel("People")
plt.show()

### Conclusions and final words

### Proposed challenges

#### Make R₀ time dependent


#### Variable death rate



#### Stochasticity of the parameters



#### Add real data

